In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from constants.companies import get_company_by_code
company = get_company_by_code("GL")
company_id = company.company_id

In [ ]:
from tofu.data import get_forecast_start_from_db
forecast_start_year, forecast_start_week = get_forecast_start_from_db(
    company_id=company.company_id,
)

In [ ]:
from tofu.app.section_download_data import section_download_data
df_order_history, df_calendar, df_latest_forecasts, df_estimations = section_download_data(
    company_id=company.company_id
)

In [8]:
num_weeks = 11

In [ ]:
df_order_history.tail()

In [ ]:
df_latest_forecasts

In [ ]:
from tofu.analysis.preprocessing import preprocess_data
df_known, df_future = preprocess_data(
    df_order_history=df_order_history,
    df_calendar=df_calendar,
    forecast_start_year=forecast_start_year,
    forecast_start_week=forecast_start_week,
    num_weeks=num_weeks + 2,
)

# Orders

In [ ]:
from tofu.analysis.order_history_analysis import create_total_orders_mapping_table, prepare_data_for_mapping
from tofu.configs import holidays_to_match_dict

country = company.country

df_concat_with_holidays = prepare_data_for_mapping(df_known=df_known, df_future=df_future, country=country)
holiday_to_match_list = holidays_to_match_dict[country]
df_mapped = create_total_orders_mapping_table(
    df_with_holidays=df_concat_with_holidays, num_years_to_map=1, holidays_to_match=holiday_to_match_list
)

In [ ]:
from tofu.analysis.order_history_analysis import add_growth_projections, add_projected_orders
df_known_mapped, df_future_mapped = add_growth_projections(
    df_mapped=df_mapped,
    df_known=df_known,
    df_future=df_future
)

df_future_mapped, projected_order_columns = add_projected_orders(
    df_future_mapped=df_future_mapped,
    growth_rate_columns=[
        "growth_rate_week_to_week",
        "growth_rate_rolling_4_weeks",
        "growth_rate_rolling_12_weeks",
        "growth_rate_week_to_week_linear_projection",
    ],
)
df_future_mapped = df_future_mapped.head(num_weeks)

# Estimations

In [ ]:
from catalog_connector import connection

df_estimations = connection.sql(
    f"""
        select
            *
        from dev.`~sylvia_liu_mlgold`.tofu_estimations
        where company_id = '{company_id}'
    """
).toPandas()

In [ ]:
df_estimations.head()

In [ ]:
from tofu.analysis.estimations_analysis import get_latest_estimations
df_latest_estimations = get_latest_estimations(
    df_estimations=df_estimations,
    df_future_mapped=df_future_mapped
)

df_latest_estimations

In [ ]:
from tofu.analysis.estimations_analysis import get_estimations_pivot_table
df_estimations_pivoted, weeks_to_include = get_estimations_pivot_table(
    df_estimations=df_estimations,
    df_future_mapped=df_future_mapped,
    df_known_mapped=df_known_mapped,
    num_weeks_known_to_include=10,
)

In [ ]:
from tofu.analysis.estimations_analysis import get_estimations_delta_table

df_estimations_delta = get_estimations_delta_table(
    df_estimations=df_estimations,
    df_known_mapped=df_known_mapped,
    df_latest_estimations=df_latest_estimations,
    weeks_to_include=weeks_to_include,
)

df_estimations_delta

# Flex share

In [ ]:
from tofu.analysis.flex_share_analysis import add_flex_development_projections

df_known_mapped_with_flex, df_future_mapped_with_flex = add_flex_development_projections(
    df_mapped=df_mapped,
    df_known=df_known_mapped,
    df_future=df_future_mapped
)


In [ ]:
from tofu.analysis.flex_share_analysis import add_projected_flex_share


df_future_mapped_with_flex, projected_flex_share_columns = add_projected_flex_share(
    df_future_mapped=df_future_mapped_with_flex,
    flex_share_development_columns=[
        "flex_share_development_week_to_week",
        "flex_share_development_rolling_4_weeks",
        "flex_share_development_rolling_12_weeks",
    ],
)

In [ ]:
df_known_mapped_with_flex

In [ ]:
df_future_mapped.columns

In [13]:
df_future_mapped["analysts_forecast_total_orders"] = df_future_mapped["orders_based_on_growth_rate_rolling_4_weeks"]
df_future_mapped_with_flex["analysts_forecast_flex_share"] = df_future_mapped_with_flex["projected_based_on_flex_share_development_week_to_week"]

In [ ]:
from tofu.analysis.postprocessing import calculate_forecasts

df_forecast = calculate_forecasts(
    df_future_mapped=df_future_mapped,
    df_future_mapped_with_flex=df_future_mapped_with_flex,
)

df_forecast.head()

In [15]:
from tofu.configs import forecast_group_ids, forecast_model_id, forecast_job_id

In [16]:
from tofu.analysis.postprocessing import prepare_run_metadata
from datetime import datetime
import pytz
import uuid
forecast_job_run_id = str(uuid.uuid4()).upper()
created_at = datetime.now(pytz.utc)

df_run_metadata = prepare_run_metadata(
    df_forecast=df_forecast,
    company_id=company_id,
    forecast_start_year=forecast_start_year,
    forecast_start_week=forecast_start_week,
    forecast_model_id=forecast_model_id,
    forecast_group_ids=forecast_group_ids,
    forecast_job_id=forecast_job_id,
    forecast_job_run_id=forecast_job_run_id,
    forecast_horizon=num_weeks,
    created_at=created_at,
)

In [17]:
from tofu.analysis.postprocessing import prepare_job_run_data, prepare_forecast_orders_data
df_forecast_job_runs = prepare_job_run_data(df_run_metadata=df_run_metadata)
df_forecast_orders = prepare_forecast_orders_data(df_forecast=df_forecast, df_run_metadata=df_run_metadata, forecast_group_ids=forecast_group_ids)

In [ ]:
df_forecast_orders

In [ ]:
df_forecast_orders

In [20]:
# from tofu.data import append_pandas_df_to_catalog

# append_pandas_df_to_catalog(
#     df=df_forecast_orders,
#     table_name="forecast_orders",
#     schema="forecasting",
# )

In [21]:
# append_pandas_df_to_catalog(
#     df=df_run_metadata,
#     table_name="forecast_job_run_metadata",
#     schema="forecasting",
# )

In [22]:

# append_pandas_df_to_catalog(
#     df=df_forecast_job_runs,
#     table_name="forecast_job_runs",
#     schema="forecasting",
# )


# Estimations

In [ ]:
df_order_history.head()

In [ ]:
df_estimations_known = df_estimations.merge(df_order_history, on=["menu_year", "menu_week"], how="inner")
df_estimations_known["retention_rate_orders"] = df_estimations_known["total_orders"] / df_estimations_known["total_orders_estimated"]
df_estimations_known[df_estimations_known["num_days_before_cutoff"] == 21].tail(60)

In [ ]:
df_forecast

In [ ]:
df_tmp = df_forecast[["menu_year", "menu_week"]].drop_duplicates().merge(
    df_estimations, on=["menu_year", "menu_week"], how="inner",
)

df_tmp.sort_values(
    by=["menu_year", "menu_week", "num_days_before_cutoff"],
    ascending=[True, True, True],
)

In [ ]:
df_estimations[df_estimations["estimation_generated_at"] == df_estimations["estimation_generated_at"].max()]

In [ ]:
df_forecast.head()